In [ ]:
%matplotlib inline

%pylab inline
pylab.rcParams['figure.figsize'] = (12,10)

import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

#data_path = '/Users/ryoungblood/opencv_tests/data/'
#data_path = '/Users/ryoungblood/image_similarity/original_images/'
data_path = '/Users/ryoungblood/image_similarity/kris_images/Winchester_small/'

In [ ]:
img = cv2.imread(os.path.join(data_path,'IMG_6121.JPG_resizeXpct.jpg'))
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [ ]:
#cv2.imwrite(os.path.join(data_path,'boot2.png'),img)

In [ ]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(img_gray, cmap='gray')

In [ ]:
img_negate = 255 - img_gray
plt.imshow(img_negate, cmap='gray')

In [ ]:
# Get the horizontal Sobel
img_sobel_h_64f = cv2.Sobel(img_negate,cv2.CV_64F,1,0,ksize=3)
img_sobel_h_64f_abs = np.absolute(img_sobel_h_64f)
img_sobel_h_8u = np.uint8(img_sobel_h_64f_abs)

# Get the vertical Sobel
img_sobel_v_64f = cv2.Sobel(img_negate,cv2.CV_64F,0,1,ksize=3)
img_sobel_v_64f_abs = np.absolute(img_sobel_v_64f)
img_sobel_v_8u = np.uint8(img_sobel_v_64f_abs)

# Bitwise OR those two
img_sobel = cv2.bitwise_or(img_sobel_h_8u,img_sobel_v_8u)
print(img_sobel[:10,:10])

plt.imshow(img_sobel, cmap='gray')

In [ ]:
img_gauss = cv2.GaussianBlur(img_sobel, (9,9), 0)
plt.imshow(img_gauss, cmap='gray')

In [ ]:
print(img_gauss.shape)
print(img_gauss[:10,:10])

In [ ]:
threshold = 50

ret,img_binary = cv2.threshold(img_gauss,threshold,255,cv2.THRESH_BINARY)
ret,img_binary_inv = cv2.threshold(img_gauss,threshold,255,cv2.THRESH_BINARY_INV)
ret,img_trunc = cv2.threshold(img_gauss,threshold,255,cv2.THRESH_TRUNC)
ret,img_tozero = cv2.threshold(img_gauss,threshold,255,cv2.THRESH_TOZERO)
ret,img_tozero_inv = cv2.threshold(img_gauss,threshold,255,cv2.THRESH_TOZERO_INV)

titles = ['Original Image','BINARY','BINARY_INV','TRUNC','TOZERO','TOZERO_INV']
images = [img_gauss, img_binary, img_binary_inv, img_trunc, 
          img_tozero, img_tozero_inv]

for i in range(6):
    plt.subplot(2,3,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])

plt.show()

#plt.imshow(binary, cmap='gray')


In [ ]:
print('TOZERO')
plt.imshow(img_tozero, cmap='gray')

In [ ]:
img_tozero_bgr = cv2.cvtColor(img_tozero, cv2.COLOR_GRAY2BGR)
plt.imshow(img_tozero_bgr)

In [ ]:
mask = np.zeros((img_tozero.shape[0]+2,img_tozero.shape[1]+2),np.uint8)
diff = (6,6,6)
cv2.floodFill(image=img_tozero_bgr,mask=mask,seedPoint=(0,0),
             newVal=(255,0,255),loDiff=diff,upDiff=diff, flags=4)
cv2.floodFill(image=img_tozero_bgr,mask=mask,seedPoint=(img_tozero.shape[1]-1,0),
             newVal=(255,0,255),loDiff=diff,upDiff=diff, flags=4)
cv2.floodFill(image=img_tozero_bgr,mask=mask,seedPoint=(img_tozero.shape[1]-1,img_tozero.shape[0]-1),
             newVal=(255,0,255),loDiff=diff,upDiff=diff, flags=4)

cv2.floodFill(image=img_tozero_bgr,mask=mask,seedPoint=(0,img_tozero.shape[0]-1),
             newVal=(255,0,255),loDiff=diff,upDiff=diff, flags=4)
plt.imshow(img_tozero_bgr)

In [ ]:
plt.imshow(mask, cmap='gray')

In [ ]:
background_pass_mask = cv2.cvtColor(mask[1:-1,1:-1]*255,cv2.COLOR_GRAY2BGR)

In [ ]:
foreground_pass_mask = bitwise_not(background_pass_mask)

In [ ]:
new_bg_color = (0,255,255)
new_bg = np.full(img.shape,new_bg_color,dtype=uint8)
background = bitwise_and(new_bg,background_pass_mask)
plt.imshow(background)

In [ ]:
foreground = bitwise_and(img,foreground_pass_mask)
plt.imshow(foreground)

In [ ]:
output_img = foreground + background
plt.imshow(output_img)